# Spot aberration correction example

This notebook shows how to use SLMcontroller for aberration compensation based on https://www.nature.com/articles/nphoton.2010.85

In [1]:
import json
import numpy as np
import time

import requests

In [2]:
class SLM():
    def __init__(self,server_url):
        self._server_url = server_url
        # self.phase = 0
        # self.probZone = 0
        # self.refZone = 0
        # self.phaseStep = 0
        # self.IDsList = np.array([])

    @property
    def phase(self):
        self.get_from_server(self._server_url + "optimiser/phase" )
        return self.__phase
    
    @phase.setter
    def phase(self, phase):
        self.get_from_server(self._server_url + "optimiser/phase/" + str(float(phase)))

    @property
    def phaseStep(self):
        self.get_from_server(self._server_url + "optimiser/phaseStep" )
        return self.__phaseStep
    
    @phaseStep.setter
    def phaseStep(self, phaseStep):
        self.get_from_server(self._server_url + "optimiser/phaseStep/" + str(float(phaseStep)))

    @property
    def IDsList(self):
        self.get_from_server(self._server_url + "optimiser/IDsList")
        return self.__IDsList
    
    @IDsList.setter
    def IDsList(self,list):
        self.get_from_server(self._server_url + "optimiser/IDsList")

    @property
    def probZone(self):
        self.get_from_server(self._server_url + "optimiser/probzone")
        return self.__probZone
    
    @probZone.setter
    def probZone(self,probZone):
        if probZone not in self.IDsList:
            raise ValueError('Probe zone is not in the range of available zones. Try reloading the list with loadIDsList() or check your server configuration.')
            return
        self.get_from_server(self._server_url + "optimiser/probzone/" + str(int(probZone)))

    @property
    def refZone(self):
        self.get_from_server(self._server_url + "optimiser/refzone")
        return self.__refZone
    
    @refZone.setter
    def refZone(self,refZone):
        if refZone not in self.IDsList:
            raise ValueError('Reference zone is not in the range of available zones. Try reloading the list with loadIDsList() or check your server configuration.')
            return
        self.get_from_server(self._server_url + "optimiser/refzone/" + str(int(refZone)))

    def cleanResponse(self, response):
        if response == 'None' or response == None:
            return None
        else:
            return response

    def get_from_server(self, url):
        response = requests.get(url)
        
        if response.status_code != 200:
            raise ValueError('There is a problem connecting to the server. Check your connection')
            return -1
        
        json_response = json.loads(response.text)

        if 'Phase' in json_response:
            self.__phase = self.cleanResponse(json_response['Phase'])

        if 'ProbZone' in json_response:
            self.__probZone = self.cleanResponse(json_response['ProbZone'])

        if 'RefZone' in json_response:
            self.__refZone = self.cleanResponse(json_response['RefZone'])

        if 'PhaseStep' in json_response:
            self.__phaseStep = self.cleanResponse(json_response['PhaseStep'])

        if 'IDsList' in json_response:
            if isinstance(json_response['IDsList'], (list, tuple)):
                self.__IDsList = np.copy(json_response['IDsList'])
            else:
                self.__IDsList = self.cleanResponse(json_response['IDsList'])


    def nextStep(self):
        self.get_from_server(self._server_url + "optimiser/nextStep")
        if self.phase == None:
            return False
        else:
            return True

    def start(self):
        self.get_from_server(self._server_url + "optimiser/start")

In [3]:
SLM1030 = SLM('http://192.168.2.2:5000/')

In [8]:
# Data acquisition sequence (misses camera functions)

SLM1030 = SLM('http://192.168.2.2:5000/')
SLM1030.refZone = 20
SLM1030.IDsList
SLM1030.probZone = SLM1030.IDsList[0]
SLM1030.phaseStep = 10

for ID in SLM1030.IDsList:
    if ID != SLM1030.refZone:
        SLM1030.probZone = ID
        SLM1030.phase = 0
        SLM1030.start()
        while SLM1030.nextStep():
            time.sleep(0.1)
            print("probZone: ", SLM1030.probZone, " refZone: ", SLM1030.refZone, " phase: ", SLM1030.phase)

probZone:  16  refZone:  20  phase:  80.0
probZone:  16  refZone:  20  phase:  90.0
probZone:  16  refZone:  20  phase:  100.0
probZone:  16  refZone:  20  phase:  110.0
probZone:  16  refZone:  20  phase:  120.0
probZone:  16  refZone:  20  phase:  130.0
probZone:  16  refZone:  20  phase:  140.0
probZone:  16  refZone:  20  phase:  150.0
probZone:  16  refZone:  20  phase:  160.0
probZone:  16  refZone:  20  phase:  170.0
probZone:  16  refZone:  20  phase:  180.0
probZone:  16  refZone:  20  phase:  190.0
probZone:  16  refZone:  20  phase:  200.0
probZone:  16  refZone:  20  phase:  210.0
probZone:  16  refZone:  20  phase:  220.0
probZone:  16  refZone:  20  phase:  230.0
probZone:  16  refZone:  20  phase:  240.0
probZone:  16  refZone:  20  phase:  250.0
probZone:  16  refZone:  20  phase:  None
probZone:  17  refZone:  20  phase:  10.0
probZone:  17  refZone:  20  phase:  20.0
probZone:  17  refZone:  20  phase:  30.0
probZone:  17  refZone:  20  phase:  40.0
probZone:  17  ref

In [127]:


SLM1030.nextStep()

False

In [51]:
response.status_code

200

In [18]:
response = requests.get('http://192.168.2.2:5000/optimiser/phaseStep/15.0')
json_response = json.loads(response.text)

In [66]:
response = requests.get('http://192.168.2.2:5000/optimiser/nextStep')
json_response = json.loads(response.text)

In [67]:
json_response

{'Phase': None, 'ProbZone': 16, 'RefZone': 19}

In [12]:
json_response['Phase']

255.0

In [15]:
response

<Response [404]>

In [22]:
int(1.2)

1